In [1]:
import itertools
from qevo import *
import qutip
import numpy as np
from IPython.display import display
from tqdm import tqdm
from collections import Counter

In [2]:
r = lambda gate_l:sorted([(i[::-1], o[::-1]) for i, o in gate_l] , key=lambda _:_[0])
cnotr_l = r(cnot_l)
cpnotr_l = r(cpnot_l)

In [3]:
letters = [''.join(_) for _ in itertools.product('ABCD', repeat=2)]
to_letter_d = dict(zip(range(16),letters))
to_index_d  = dict(zip(letters,range(16)))

In [4]:
generators = {tuple(to_index_d[_[1]] for _ in gate_l) for gate_l in [cnot_l, cphase_l, cpnot_l, cnotr_l, cpnotr_l]}

In [5]:
generators

{(0, 1, 14, 15, 9, 8, 7, 6, 5, 4, 11, 10, 12, 13, 2, 3),
 (0, 6, 5, 3, 4, 2, 1, 7, 11, 13, 14, 8, 15, 9, 10, 12),
 (0, 9, 10, 3, 7, 14, 13, 4, 8, 1, 2, 11, 15, 6, 5, 12),
 (0, 13, 2, 15, 6, 11, 4, 9, 10, 7, 8, 5, 12, 1, 14, 3),
 (0, 13, 14, 3, 7, 10, 9, 4, 11, 6, 5, 8, 12, 1, 2, 15)}

In [6]:
def c(*gate_ls):
    l, *o = gate_ls
    if o:
        r, *o = o
        lr = tuple(r[l[_]] for _ in range(16))
        return c(lr,*o)
    else:
        return l

In [7]:
def products(generators):
    return {c(l,r) for l,r in tqdm(itertools.product(generators, repeat=2), total=len(generators)**2)}

In [8]:
def repeated_generation(generators,repeat):
    print(len(generators), flush=True)
    if repeat:
        next_gen = products(generators)|generators
        return repeated_generation(next_gen, repeat-1)
    return generators

In [9]:
generated = repeated_generation(generators, 3)
generated = list(generated)

5


100%|██████████| 25/25 [00:00<00:00, 18838.95it/s]

10



100%|██████████| 100/100 [00:00<00:00, 36532.57it/s]

12



100%|██████████| 144/144 [00:00<00:00, 51267.28it/s]

12


In [10]:
r = np.array([1 if (c(l,r)==tuple(range(16))) else 0 for l,r in itertools.product(generated, repeat=2)])
r.shape = [len(generated)]*2
r

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [11]:
generated

[(0, 7, 8, 15, 9, 14, 1, 6, 10, 13, 2, 5, 3, 4, 11, 12),
 (0, 9, 5, 12, 1, 8, 4, 13, 14, 7, 11, 2, 15, 6, 10, 3),
 (0, 1, 14, 15, 9, 8, 7, 6, 5, 4, 11, 10, 12, 13, 2, 3),
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15),
 (0, 7, 11, 12, 13, 10, 6, 1, 14, 9, 5, 2, 3, 4, 8, 15),
 (0, 13, 2, 15, 6, 11, 4, 9, 10, 7, 8, 5, 12, 1, 14, 3),
 (0, 9, 10, 3, 7, 14, 13, 4, 8, 1, 2, 11, 15, 6, 5, 12),
 (0, 6, 5, 3, 4, 2, 1, 7, 11, 13, 14, 8, 15, 9, 10, 12),
 (0, 6, 10, 12, 13, 11, 7, 1, 2, 4, 8, 14, 15, 9, 5, 3),
 (0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15),
 (0, 13, 14, 3, 7, 10, 9, 4, 11, 6, 5, 8, 12, 1, 2, 15),
 (0, 4, 11, 15, 6, 2, 13, 9, 5, 1, 14, 10, 3, 7, 8, 12)]

In [12]:
def order(el):
    el_c = el
    o = 1
    while el_c!=tuple(range(16)):
        el_c = c(el_c,el)
        o += 1
    return o

In [13]:
[order(_) for _ in generated]

[3, 6, 2, 1, 2, 2, 2, 2, 3, 2, 2, 6]

In [14]:
def permutation2_to_index_map(permutation2):
    return [(_//4,_%4) for _ in permutation2]
def permute2(probs, targets, permutation2_index, N):
    slices_a = [slice(None)]*N
    slices_b = [slice(None)]*N
    t1, t2 = targets
    new_probs = probs.copy()
    for (l1, l2),(r1,r2) in zip(itertools.product(range(4), repeat=2),permutation2_index):
        slices_a[t1] = l1
        slices_a[t2] = l2
        slices_b[t1] = r1
        slices_b[t2] = r2
        new_probs[tuple(slices_b)] = probs[tuple(slices_a)]
    return new_probs
f = 0.9
p2 = 0.99
index_maps = [permutation2_to_index_map(_) for _ in generated]
def calcF(ind, meas):
    probs = get_new_probs(2, f)
    probs_perm = probs
    probs_perm = permute2(probs, [0,1], ind, 2)
    probs_dep = depolarize2(probs_perm, [0,1], p2)
    assert abs(np.sum(probs_dep)-1)<1e-9
    return np.round(measure(probs_dep, 1, [0,meas], 2, f, p2), decimals=7) # rounded to permit hashing
fids = [tuple([fidelity(calcF(ind, _)) for _ in [1,2,3]]) for ind in index_maps]

In [15]:
Counter(fids)

Counter({(0.89164140000000003, 0.91659669999999982, 0.91659669999999982): 2,
         (0.89287620000000012, 0.89287620000000012, 0.89287620000000012): 2,
         (0.91659669999999982, 0.89164140000000003, 0.91659669999999982): 2,
         (0.91659669999999982, 0.91659669999999982, 0.89164140000000003): 6})

In [16]:
gates = [qutip.controlled_gate(cgate, N=2, control=0, target=1)
         for cgate in [cnot_gate, cphase_gate, cnot_gate*cphase_gate]]
gates+= [qutip.controlled_gate(cgate, N=2, control=1, target=0)
         for cgate in [cnot_gate, cnot_gate*cphase_gate]]
gates+= [qutip.tensor(l, r)
         for l, r in itertools.product([qutip.identity(2),qutip.hadamard_transform(1),qutip.phasegate(np.pi/2)], repeat=2)]

gatesT = [qutip.tensor(l,r).permute([0,2,1,3]) for l,r in itertools.product(gates, repeat=2)]

def gates_equal(l,r):
    return True if abs((l-r).norm())<0.001 else False

def gate_is_in(gate, other_gates):
    return any(gates_equal(gate,_) for _ in other_gates)

def unique(gates):
    unique_gates = []
    gates = list(gates)
    for i, gate in tqdm(enumerate(gates), total=len(gates)):
        if not gate_is_in(gate, unique_gates):
            unique_gates.append(gate)
    return gates

print('total local Cliford gates (gen 2) (maybe not unique)', len(gatesT), flush=True)

#gatesT = unique(gatesT)
#print('total local Cliford gates (gen 2) (unique)', len(gatesT))

gatesT = [l*r for l,r in tqdm(itertools.product(gatesT, repeat=2), total=len(gatesT)**2)]
print('total local Cliford gates (gen 3) (maybe not unique)', len(gatesT), flush=True)

total local Cliford gates (gen 2) (maybe not unique) 196


100%|██████████| 38416/38416 [00:27<00:00, 1400.91it/s]

total local Cliford gates (gen 3) (maybe not unique) 38416


In [17]:
gates_letters = []
for g in tqdm(gatesT):
    l = letter_map(g, exceptions=False) # I thought it would be faster if we do not
    if l is None:                       # use exceptions... Well, while true in general,
        continue                        # this is not the inner loop, so it does not
    gates_letters.append(l)             # matter

100%|██████████| 38416/38416 [20:42<00:00, 20.10it/s]


In [18]:
len(gates_letters)

730

In [19]:
generators = {tuple(to_index_d[_[1]] for _ in gate_l) for gate_l in gates_letters}

In [20]:
len(generators)

115

In [21]:
generated = repeated_generation(generators, 2) # after 2 it repeats
generated = list(generated)

115


100%|██████████| 13225/13225 [00:00<00:00, 138878.67it/s]

2113



100%|██████████| 4464769/4464769 [00:28<00:00, 157251.33it/s]

11520


In [22]:
import pickle
#with open('permutation_subgroup.pickle','wb') as f:
#    pickle.dump(generated, f)

In [23]:
orders = [order(_) for _ in generated]

In [24]:
Counter(orders)

Counter({1: 1, 2: 375, 3: 800, 4: 2280, 5: 2304, 6: 3360, 8: 1440, 12: 960})

In [24]:
cnot_i

[((0, 1), (3, 1)),
 ((0, 3), (3, 3)),
 ((1, 0), (1, 2)),
 ((1, 1), (2, 3)),
 ((1, 3), (2, 1)),
 ((2, 0), (2, 2))]

In [25]:
l = r = qutip.controlled_gate(cnot_gate, N=2, control=0, target=1)
g = qutip.tensor(l,r).permute([0,2,1,3])
gm = letter_map(g)
gi = tuple(to_index_d[_[1]] for _ in gm)
cnot_fulli = permutation2_to_index_map(gi)
cnot_fulli

[(0, 0),
 (3, 1),
 (0, 2),
 (3, 3),
 (1, 2),
 (2, 3),
 (1, 0),
 (2, 1),
 (2, 2),
 (1, 3),
 (2, 0),
 (1, 1),
 (3, 0),
 (0, 1),
 (3, 2),
 (0, 3)]

In [26]:
permute2(get_new_probs(2, f), [0,1], cnot_fulli, 2)

array([[ 0.81      ,  0.00111111,  0.03      ,  0.00111111],
       [ 0.00111111,  0.00111111,  0.03      ,  0.00111111],
       [ 0.00111111,  0.00111111,  0.03      ,  0.00111111],
       [ 0.03      ,  0.03      ,  0.00111111,  0.03      ]])

In [27]:
permute2_by_pairs(get_new_probs(2, f), [0,1], cnot_i, 2)

array([[ 0.81      ,  0.00111111,  0.03      ,  0.00111111],
       [ 0.00111111,  0.00111111,  0.03      ,  0.00111111],
       [ 0.00111111,  0.00111111,  0.03      ,  0.00111111],
       [ 0.03      ,  0.03      ,  0.00111111,  0.03      ]])

In [28]:
index_maps = [permutation2_to_index_map(_) for _ in generated]
def calcF(ind, meas):
    probs = get_new_probs(2, f)
    probs_perm = probs
    probs_perm = permute2(probs, [0,1], ind, 2)
    probs_dep = depolarize2(probs_perm, [0,1], p2)
    assert abs(np.sum(probs_dep)-1)<1e-9
    return np.round(measure(probs_dep, 1, [0,meas], 2, f, p2), decimals=7) # rounded to permit hashing
fids = [tuple([fidelity(calcF(ind, _)) for _ in [1,2,3]]) for ind in index_maps]

In [29]:
Counter(fids)

Counter({(0.0061230999999999985,
          0.0061230999999999985,
          0.035707999999999997): 432,
         (0.0061230999999999985,
          0.035707999999999997,
          0.0061230999999999985): 432,
         (0.0061230999999999985,
          0.056738699999999996,
          0.22129830000000003): 432,
         (0.0061230999999999985,
          0.22129830000000003,
          0.056738699999999996): 432,
         (0.035707999999999997,
          0.0061230999999999985,
          0.0061230999999999985): 432,
         (0.035707999999999997,
          0.035707999999999997,
          0.035707999999999997): 216,
         (0.035707999999999997,
          0.056738699999999996,
          0.056738699999999996): 216,
         (0.035707999999999997, 0.22129830000000003, 0.22129830000000003): 432,
         (0.036942800000000005,
          0.071156999999999984,
          0.071156999999999984): 216,
         (0.036942800000000005, 0.22895199999999996, 0.22895199999999996): 216,
         (0.056738